In [1]:
import csv
import re
from io import BytesIO
import requests
import pdfplumber
import os
import pikepdf
import pandas as pd
import tabula
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader, PdfWriter
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import Choropleth
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from fuzzywuzzy import process
import geopandas as gpd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

## 0. Leitura dos Ficheiros

In [2]:
# Guardar todos os outputs numa única pasta
# output_folder = Path.cwd() / "output"
# output_folder.mkdir(parents=True, exist_ok=True)

# Exemplo: mapa.save(output_folder / 'mapa_de_calor_portugal_Danielle.html')

In [3]:
p = Path.cwd()
data_folder = p.parent.parent / "data"
shape_folder = p.parent.parent / "shape"

## 1. Incêndios

In [4]:
file_path = data_folder / "Dados Incêndios Urbanos por Freguesia 2014-2021 ISCTE.xlsx"
Fogo = pd.read_excel(file_path)
Fogo

,Ano,Mês,Distrito,CIM,DICO,Concelho,DICOFRE,Freguesia,CodNatureza,Natureza,Ocorrências,MeiosTerrestres,Duração Minutos
0,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10119,Valongo do Vouga,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,1.0,61.0
1,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10121,Águeda e Borralha,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,1.0,27.0
2,2014,1.0,Aveiro,Região de Aveiro,102.0,Albergaria-a-Velha,10209,Albergaria-a-Velha e Valmaior,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,4.0,259.0
3,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10312,Vila Nova de Monsarros,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,1.0,48.0
4,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10313,Vilarinho do Bairro,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,2.0,147.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44439,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182321,Ranhados,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,3.0,110.0
44440,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182338,Faíl e Vila Chã de Sá,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,2.0,68.0
44441,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182341,Viseu,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,3.0,7.0,207.0
44442,2021,12.0,Viseu,Viseu Dão Lafões,1824.0,Vouzela,182414,Fataunços e Figueiredo Das Donas,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,2.0,140.0


In [5]:
Fogo["Natureza"] = Fogo["Natureza"].str.split('/').str[-1].str.strip()
Fogo.dropna(subset=["Concelho"], inplace=True)
Fogo

,Ano,Mês,Distrito,CIM,DICO,Concelho,DICOFRE,Freguesia,CodNatureza,Natureza,Ocorrências,MeiosTerrestres,Duração Minutos
0,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10119,Valongo do Vouga,2101.0,Habitacional,1.0,1.0,61.0
1,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10121,Águeda e Borralha,2101.0,Habitacional,1.0,1.0,27.0
2,2014,1.0,Aveiro,Região de Aveiro,102.0,Albergaria-a-Velha,10209,Albergaria-a-Velha e Valmaior,2101.0,Habitacional,1.0,4.0,259.0
3,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10312,Vila Nova de Monsarros,2101.0,Habitacional,1.0,1.0,48.0
4,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10313,Vilarinho do Bairro,2101.0,Habitacional,1.0,2.0,147.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44438,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182301,Abraveses,2101.0,Habitacional,3.0,7.0,171.0
44439,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182321,Ranhados,2101.0,Habitacional,1.0,3.0,110.0
44440,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182338,Faíl e Vila Chã de Sá,2101.0,Habitacional,1.0,2.0,68.0
44441,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182341,Viseu,2101.0,Habitacional,3.0,7.0,207.0


In [6]:
pivot_table = Fogo.pivot_table(Fogo, 
                             index=['Ano', 'Mês', 'Concelho'],
                             columns='Natureza',               
                             aggfunc='size',                   
                             fill_value=0)                     

pivot_table

Natureza                      Bibliotecas e Arquivos  Desporto e Lazer  \
Ano  Mês  Concelho                                                       
2014 1.0  Abrantes                                 0                 0   
          Aguiar da Beira                          0                 0   
          Albergaria-a-Velha                       0                 0   
          Albufeira                                0                 0   
          Alcanena                                 0                 0   
...                                              ...               ...   
2021 12.0 Vouzela                                  0                 0   
          Águeda                                   0                 0   
          Évora                                    0                 0   
          Ílhavo                                   0                 0   
          Óbidos                                   0                 0   

Natureza                      Edifícios degradados ou devolutos  \
Ano  Mês  Concelho                                                
2014 1.0  Abrantes                                            0   
          Aguiar da Beira                                     0   
          Albergaria-a-Velha                                  0   
          Albufeira                                           0   
          Alcanena                                            0   
...                                                         ...   
2021 12.0 Vouzela                                             0   
          Águeda                                              0   
          Évora                                               0   
          Ílhavo                                              0   
          Óbidos                                              0   

Natureza                      Espectáculos e Reuniões Públicas  \
Ano  Mês  Concelho                                               
2014 1.0  Abrantes                                           0   
          Aguiar da Beira                                    0   
          Albergaria-a-Velha                                 0   
          Albufeira                                          0   
          Alcanena                                           0   
...                                                        ...   
2021 12.0 Vouzela                                            0   
          Águeda                                             0   
          Évora                                              0   
          Ílhavo                                             0   
          Óbidos                                             0   

Natureza                      Estacionamento de superfície  \
Ano  Mês  Concelho                                           
2014 1.0  Abrantes                                       0   
          Aguiar da Beira                                0   
          Albergaria-a-Velha                             0   
          Albufeira                                      0   
          Alcanena                                       0   
...                                                    ...   
2021 12.0 Vouzela                                        0   
          Águeda                                         0   
          Évora                                          0   
          Ílhavo                                         0   
          Óbidos                                         0   

Natureza                      Estacionamento em profundidade ou silo  \
Ano  Mês  Concelho                                                     
2014 1.0  Abrantes                                                 0   
          Aguiar da Beira                                          0   
          Albergaria-a-Velha                                       0   
          Albufeira                                                0   
          Alcanena                                                 

In [7]:
FogoPosto = Fogo.groupby(["Ano", "Mês", "Concelho"])[["Ocorrências", "MeiosTerrestres", "Duração Minutos"]].sum()
FogoPosto

Ocorrências  MeiosTerrestres  Duração Minutos
Ano  Mês  Concelho                                                         
2014 1.0  Abrantes                    1.0              1.0             44.0
          Aguiar da Beira             1.0              2.0             85.0
          Albergaria-a-Velha          1.0              4.0            259.0
          Albufeira                   1.0              3.0            107.0
          Alcanena                    3.0              4.0            133.0
...                                   ...              ...              ...
2021 12.0 Vouzela                     1.0              2.0            140.0
          Águeda                      1.0              1.0             39.0
          Évora                       2.0              7.0            327.0
          Ílhavo                      1.0              6.0            521.0
          Óbidos                      2.0              5.0            438.0

[16686 rows x 3 columns]

In [8]:
FogoFinal = pd.merge(pivot_table, FogoPosto, how='inner', on=['Ano', 'Mês', 'Concelho'])
FogoFinal

Bibliotecas e Arquivos  Desporto e Lazer  \
Ano  Mês  Concelho                                                       
2014 1.0  Abrantes                                 0                 0   
          Aguiar da Beira                          0                 0   
          Albergaria-a-Velha                       0                 0   
          Albufeira                                0                 0   
          Alcanena                                 0                 0   
...                                              ...               ...   
2021 12.0 Vouzela                                  0                 0   
          Águeda                                   0                 0   
          Évora                                    0                 0   
          Ílhavo                                   0                 0   
          Óbidos                                   0                 0   

                              Edifícios degradados ou devolutos  \
Ano  Mês  Concelho                                                
2014 1.0  Abrantes                                            0   
          Aguiar da Beira                                     0   
          Albergaria-a-Velha                                  0   
          Albufeira                                           0   
          Alcanena                                            0   
...                                                         ...   
2021 12.0 Vouzela                                             0   
          Águeda                                              0   
          Évora                                               0   
          Ílhavo                                              0   
          Óbidos                                              0   

                              Espectáculos e Reuniões Públicas  \
Ano  Mês  Concelho                                               
2014 1.0  Abrantes                                           0   
          Aguiar da Beira                                    0   
          Albergaria-a-Velha                                 0   
          Albufeira                                          0   
          Alcanena                                           0   
...                                                        ...   
2021 12.0 Vouzela                                            0   
          Águeda                                             0   
          Évora                                              0   
          Ílhavo                                             0   
          Óbidos                                             0   

                              Estacionamento de superfície  \
Ano  Mês  Concelho                                           
2014 1.0  Abrantes                                       0   
          Aguiar da Beira                                0   
          Albergaria-a-Velha                             0   
          Albufeira                                      0   
          Alcanena                                       0   
...                                                    ...   
2021 12.0 Vouzela                                        0   
          Águeda                                         0   
          Évora                                          0   
          Ílhavo                                         0   
          Óbidos                                         0   

                              Estacionamento em profundidade ou silo  \
Ano  Mês  Concelho                                                     
2014 1.0  Abrantes                                                 0   
          Aguiar da Beira                                          0   
          Albergaria-a-Velha                                       0   
          Albufeira                                                0   
          Alcanena                                                 0   
...                      

## 2. Densidade Populacional

In [9]:
file_path = data_folder / "PORDATA_Densidade-populacional.xlsx"
Denso = pd.read_excel(file_path, header=11, usecols=range(17))
Denso.drop("Âmbito Geográfico", axis=1, inplace=True)
Denso = Denso.dropna()
Denso.drop(334, inplace=True)
Denso.rename(columns={'Anos': "Concelho"}, inplace=True)
Denso["Concelho"] = Denso["Concelho"].str.title()
Denso

,Concelho,2001,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos De Valdevez,55.2,52,51.5,50.6,49.7,49,48.4,47.9,47.4,47.1,46.8,46.6,46.5,46.4,46.2
1,Caminha,124.1,123.4,122.9,122.5,121.9,120.9,119.7,118.5,117.5,116.8,116.3,116.1,116.2,116.6,117.4
2,Melgaço,41.9,39.4,39,38.3,37.5,36.9,36.3,35.8,35.3,34.7,34.1,33.7,33.2,32.5,32.1
3,Monção,94.3,92,91.5,90.8,89.9,88.9,87.8,86.9,86.1,85.5,85,84.7,84.8,84.7,84.6
4,Paredes De Coura,69.1,67.2,66.9,66.1,65.3,64.8,64.2,63.8,63.4,63,62.8,62.8,63,63,62.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,193,204.3,204.6,204.6,203.5,201.2,198.1,195,192.9,192,192.3,193.7,194.8,195.3,196.2
304,Santa Cruz,448.9,505,520.4,528.3,527.2,523.7,519.1,514.5,511.5,510.4,510.7,512.3,518,524.7,531.2
305,Santana,91.2,83.1,81.6,80.1,78.5,77,75.4,73.8,72.4,71.2,70.4,69.9,69.4,68.6,68
306,São Vicente,76.5,74.1,73.2,71.7,70,68.5,66.6,65,63.8,62.8,62.1,61.7,61.8,61.8,61.7


## 3. População Residente

In [10]:
file_path = data_folder / "PORDATA_NUTS2013_Total.xlsx"
Viver = pd.read_excel(file_path, header=11, usecols=range(25))
Viver.drop("Âmbito Geográfico", axis=1, inplace=True)
Viver = Viver.dropna()
Viver.drop(333, inplace=True)
Viver.rename(columns={'Anos': "Concelho"}, inplace=True)
Viver["Concelho"] = Denso["Concelho"].str.title()
Viver

,Concelho,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos De Valdevez,0,24686,24603,24448,24259,24062,23854,23692,23519,...,21944,21664,21420,21227,21076,20926,20844,20826,20754,20693
1,Caminha,0,17055,17100,17088,17051,17013,16986,16959,16892,...,16500,16345,16177,16045,15940,15873,15848,15863,15917,16033
2,Melgaço,0,9971,9949,9889,9796,9715,9630,9538,9465,...,8785,8652,8531,8418,8279,8136,8029,7901,7755,7648
3,Monção,0,19925,19950,19919,19843,19773,19698,19615,19536,...,18782,18556,18361,18199,18063,17953,17890,17909,17898,17868
4,Paredes De Coura,0,9565,9556,9509,9459,9429,9404,9368,9321,...,8952,8869,8811,8761,8706,8677,8685,8711,8703,8676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,0,12523,12683,12827,12971,13087,13165,13247,13316,...,13158,12969,12765,12625,12570,12585,12679,12749,12788,12841
304,Santa Cruz,0,30301,31734,33120,34466,35805,37154,38512,39847,...,42685,42307,41932,41690,41602,41625,41753,42115,42666,43190
305,Santana,0,8750,8685,8605,8511,8395,8276,8169,8062,...,7357,7204,7049,6911,6800,6722,6677,6624,6554,6490
306,São Vicente,0,6179,6174,6138,6078,6020,5979,5946,5904,...,5398,5255,5124,5027,4949,4897,4867,4872,4875,4861


## 4. Setor de Atividade

### 4.1. Primário

In [11]:
file_path = data_folder / "SetorDeAtividade.xlsx"
WorkP = pd.read_excel(file_path, header=11, usecols=[0,1,6,7])
WorkP.drop("Âmbito Geográfico", axis=1, inplace=True)
WorkP = WorkP.dropna()
WorkP.drop(333, inplace=True)
WorkP.rename(columns={'Anos': "Concelho"}, inplace=True)
WorkP["Concelho"] = WorkP["Concelho"].str.title()
WorkP

,Concelho,2011,2021
0,Arcos De Valdevez,514,374
1,Caminha,316,213
2,Melgaço,258,193
3,Monção,377,432
4,Paredes De Coura,185,151
...,...,...,...
303,Ribeira Brava,156,119
304,Santa Cruz,439,318
305,Santana,410,229
306,São Vicente,100,105


### 4.2. Secundário

In [12]:
file_path = data_folder / "SetorDeAtividade.xlsx"
WorkS = pd.read_excel(file_path, header=11, usecols=[0,1,12,13])
WorkS.drop("Âmbito Geográfico", axis=1, inplace=True)
WorkS = WorkS.dropna()
WorkS.drop(333, inplace=True)
WorkS.rename(columns={'2011.1': 2011}, inplace=True)
WorkS.rename(columns={'2021.1': 2021}, inplace=True)
WorkS.rename(columns={'Anos': "Concelho"}, inplace=True)
WorkS["Concelho"] = WorkS["Concelho"].str.title()
WorkS

,Concelho,2011,2021
0,Arcos De Valdevez,2369,2486
1,Caminha,1561,1634
2,Melgaço,595,579
3,Monção,2062,2058
4,Paredes De Coura,1253,1513
...,...,...,...
303,Ribeira Brava,1327,994
304,Santa Cruz,2777,2260
305,Santana,438,348
306,São Vicente,449,293


### 4.3. Setor Terciário

In [13]:
file_path = data_folder / "SetorDeAtividade.xlsx"
WorkT = pd.read_excel(file_path, header=11, usecols=[0,1,18,19])
WorkT.drop("Âmbito Geográfico", axis=1, inplace=True)
WorkT = WorkT.dropna()
WorkT.drop(333, inplace=True)
WorkT.rename(columns={'2011.2': 2011}, inplace=True)
WorkT.rename(columns={'2021.2': 2021}, inplace=True)
WorkT.rename(columns={'Anos': "Concelho"}, inplace=True)
WorkT["Concelho"] = WorkT["Concelho"].str.title()
WorkT

,Concelho,2011,2021
0,Arcos De Valdevez,4175,4141
1,Caminha,4386,4438
2,Melgaço,1767,1657
3,Monção,4077,4182
4,Paredes De Coura,1910,1750
...,...,...,...
303,Ribeira Brava,3454,3441
304,Santa Cruz,16902,16317
305,Santana,1855,1722
306,São Vicente,1351,1281


## 5. Indíce de Envelhecimento

In [14]:
file_path = data_folder / "IndiceEnvelhecimento.xlsx"
Velho = pd.read_excel(file_path, header=11, usecols=range(25))
Velho.drop("Âmbito Geográfico", axis=1, inplace=True)
Velho.dropna(inplace=True)
Velho.drop(333, inplace=True)
Velho.rename(columns={'Anos': "Concelho"}, inplace=True)
Velho["Concelho"] = Velho["Concelho"].str.title()
Velho

,Concelho,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos De Valdevez,0,208.1,216.4,223.7,231.3,238.7,244,248.3,253.1,...,273.1,282.8,296,310.1,324.8,339.6,355.1,369.2,379.3,391.6
1,Caminha,0,138.1,141.1,145,147.3,149.9,153.1,156.4,161.3,...,203.4,209.8,218.9,229.2,239.7,247.6,253.3,258.9,265.8,272.3
2,Melgaço,0,297.4,305.3,311.1,321.1,332.3,348.2,367.2,376.7,...,393.4,410.7,433.9,451.1,473.6,501,526.9,548.2,563.3,577
3,Monção,0,209.7,217.6,222.3,227.2,234.8,242,246.3,246.3,...,277.2,284.3,292.3,301,310.2,318.2,324.1,332.2,343.5,351.5
4,Paredes De Coura,0,204.6,208.8,211.6,214.2,216.5,218.1,216.2,213.4,...,219,226,232.5,237.7,244.3,250.3,256.3,267.3,278.4,289.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,0,80.9,81,78.5,76.5,76.1,76,76,76.9,...,96.4,102.2,108,114.8,122.5,131.4,141.7,152,162.1,169.5
304,Santa Cruz,0,56.1,55.4,54.4,53.1,52.1,51.2,50.4,49.7,...,57.9,60.8,63.9,67.8,72.7,78.1,83.8,89.9,95.7,101.1
305,Santana,0,128.1,136.8,143.8,149,151.7,153.7,159.1,165.3,...,189.7,201.1,211.2,222.4,234.8,242.8,253,265.4,279.7,290.4
306,São Vicente,0,121.5,127,131.1,133.7,136.7,141.1,146,152,...,169.4,180.6,190.5,197.8,206,215.3,228.4,243.7,258.3,273.9


## 6. Zonas Rurais

In [15]:
file_path = data_folder / "ZonasCount.csv"
ZonasRurais = pd.read_csv(file_path)
ZonasRurais["Rural"] = ZonasRurais["Zona Rural"] / (ZonasRurais["Zona Urbana"] + ZonasRurais["Zona Rural"])
ZonasRurais

,Distrito,Concelho,Zona Rural,Zona Urbana,Rural
0,Aveiro,Albergaria-A-Velha,6,0,1.00
1,Aveiro,Anadia,10,0,1.00
2,Aveiro,Arouca,16,0,1.00
3,Aveiro,Aveiro,0,10,0.00
4,Aveiro,Castelo De Paiva,6,0,1.00
...,...,...,...,...,...
273,Évora,Reguengos De Monsaraz,4,0,1.00
274,Évora,Vendas Novas,2,0,1.00
275,Évora,Viana Do Alentejo,3,0,1.00
276,Évora,Vila Viçosa,4,0,1.00


## 7. IPMA

In [16]:
# Caminho do arquivo Excel
file_path = data_folder / "Dados_diarios_T_Vento_Prec_capitaisDistrito_2013_a_2023.xlsx"

# Lista de nomes dos distritos
distritos = ['Viana do Castelo', 'Braga', 'Vila Real', 'Bragança', 'Porto', 'Aveiro', 'Viseu', 'Guarda',
             'Coimbra', 'Castelo Branco', 'Leiria', 'Santarém', 'Portalegre', 'Lisboa', 'Setúbal', 'Évora',
             'Beja', 'Faro']

# Lista completa de colunas a serem lidas
colunasTempo = ['Ano', 'Mês', 'Dia'] + distritos

colunas_indices = list(range(len(colunasTempo)))

# Leitura do arquivo Excel para cada tabela
T_MED = pd.read_excel(file_path, sheet_name=1, names=colunasTempo)
T_MAX = pd.read_excel(file_path, sheet_name=2, names=colunasTempo)
T_MIN = pd.read_excel(file_path, sheet_name=3, names=colunasTempo)
DD_MED = pd.read_excel(file_path, sheet_name=4, names=colunasTempo)
DD_FFX = pd.read_excel(file_path, sheet_name=5, names=colunasTempo)
FF_MED = pd.read_excel(file_path, sheet_name=6, names=colunasTempo)
FF_MAX = pd.read_excel(file_path, sheet_name=7, names=colunasTempo)
PR_QTD = pd.read_excel(file_path, sheet_name=8, names=colunasTempo)

tabelas = [T_MED, T_MAX, T_MIN, DD_MED, DD_FFX, FF_MED, FF_MAX, PR_QTD]
for i in tabelas:
    i.rename(columns={'Viana do Castelo': "Viana Do Castelo"}, inplace=True)

T_MED

,Ano,Mês,Dia,Viana Do Castelo,Braga,Vila Real,Bragança,Porto,Aveiro,Viseu,...,Coimbra,Castelo Branco,Leiria,Santarém,Portalegre,Lisboa,Setúbal,Évora,Beja,Faro
0,2013,1,1,9.6,9.0,8.2,7.2,10.5,10.5,7.3,...,10.3,8.0,9.9,10.4,7.9,12.3,11.9,9.4,10.2,13.6
1,2013,1,2,7.9,7.1,6.6,4.8,9.0,10.0,7.3,...,9.0,9.2,7.3,10.4,9.0,11.8,11.7,8.8,9.0,12.7
2,2013,1,3,8.6,9.2,7.6,5.9,11.5,13.4,8.3,...,11.3,9.0,7.6,10.0,9.4,10.9,9.7,7.6,8.9,13.0
3,2013,1,4,8.0,7.3,7.0,5.4,11.1,11.0,9.4,...,11.2,7.8,7.7,9.4,9.3,9.9,7.3,7.6,8.7,13.2
4,2013,1,5,6.6,5.8,6.2,5.2,9.2,9.3,9.7,...,9.5,8.3,7.8,9.4,10.2,10.5,8.0,7.9,9.8,13.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4012,2023,12,27,13.3,-990.0,2.2,0.4,11.5,-990.0,7.3,...,8.2,2.7,7.7,7.6,5.6,7.7,8.0,5.0,6.2,10.9
4013,2023,12,28,12.7,-990.0,6.9,4.6,13.6,-990.0,8.8,...,12.0,6.1,10.9,9.5,9.7,9.8,10.2,7.8,9.2,13.4
4014,2023,12,29,12.0,-990.0,9.2,7.0,12.2,-990.0,9.3,...,12.1,7.4,12.1,9.8,9.5,10.0,9.7,8.9,10.2,14.5
4015,2023,12,30,12.6,-990.0,9.7,7.8,12.6,-990.0,9.0,...,11.2,8.4,11.3,11.0,10.0,10.9,9.9,8.2,10.8,13.9


## 8. Eventos Excecionais

In [17]:
file_path = data_folder / "EventosLimpos.csv"
Quebras = pd.read_csv(file_path)
Quebras.rename(columns={'Concelho origem do incidente': "Concelho"}, inplace=True)
Quebras.reset_index(drop = True, inplace=True)
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),END (MWh),SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563","0,2135252","0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009","0,2134525","0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047","1,3991121","0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000","0,0000000","0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377","0,3217694","0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,0,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017
7315,EDPD_2017_NOV_T_63,Santa Maria da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,0,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017
7316,EDPD_2017_NOV_T_64,Santa Maria da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,0,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017
7317,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,0,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017


In [18]:
# file_path = data_folder / "eventos.parquet"
# Quebras = pd.read_parquet(file_path)
# Quebras.dropna(inplace=True)
# Quebras.reset_index(drop = True, inplace=True)
# Quebras['Ano'] = Quebras['data_do_incidente'].dt.year
# Quebras['Mês'] = Quebras['data_do_incidente'].dt.month
# Quebras['Dia'] = Quebras['data_do_incidente'].dt.day
# Quebras.rename(columns={'concelho_origem_do_incidente': "Concelho"}, inplace=True)
# Quebras["Concelho"] = Quebras["Concelho"].str.title()
# Quebras['aprovado'] = Quebras['aprovado'].astype(int)
# mapeamento = {'BT': 1, 'MT': 2, 'AT': 3}
# Quebras['nt'] = Quebras['nt'].map(mapeamento)
# Quebras["Concelho"] = Quebras["Concelho"].replace({'Ponte de Sor': 'Ponte De Sôr'})
# Quebras

## 9. Junção dos dados

### 9.1. Distritos

In [19]:
file_path = shape_folder / "concelhos.shp"
Concelho = gpd.read_file(file_path)
Concelho
Concelho.rename(columns={'NAME_1': 'Distrito'}, inplace=True)
Concelho.rename(columns={'NAME_2': 'Concelho'}, inplace=True)
Concelho['Concelho'] = Concelho['Concelho'].str.title()
Concelho['Distrito'] = Concelho['Distrito'].str.title()

# Crie um dicionário vazio para armazenar os concelhos de cada distrito
dicionario_concelhos = {}

# Itere sobre os distritos únicos na coluna 'Distrito'
for distrito in Concelho['Distrito'].unique():
    # Filtrar os concelhos para o distrito atual
    concelhos_distrito = Concelho.loc[Concelho['Distrito'] == distrito, 'Concelho'].tolist()
    # Adicionar os concelhos ao dicionário
    dicionario_concelhos[distrito] = concelhos_distrito
dicionario_concelhos['Braga'].append('Guimarães')
dicionario_concelhos['Portalegre'].append('Ponte De Sor')
dicionario_concelhos['Faro'].append('Tavira')

In [20]:
# Substituir "da" por "Da" na coluna especificada
Quebras['Concelho'] = Quebras['Concelho'].str.replace(' da ', ' Da ')
Quebras['Concelho'] = Quebras['Concelho'].str.replace(' de ', ' De ')
Quebras['Concelho'] = Quebras['Concelho'].str.replace(' do ', ' Do ')
Quebras['Concelho'] = Quebras['Concelho'].str.replace(' dos ', ' Dos ')
Quebras['Concelho'] = Quebras['Concelho'].str.replace('-a-', '-A-')
Quebras['Concelho'] = Quebras['Concelho'].str.replace('-o-', '-O-')
Quebras['Concelho'] = Quebras['Concelho'].str.replace(' à ', ' À ')

In [21]:
def mapear_distrito(concelho):
    for distrito, concelhos in dicionario_concelhos.items():
        if concelho in concelhos:
            return distrito
    return None  # Retorna None se o concelho não for encontrado no dicionário

# Aplicar a função mapear_distrito à coluna 'Concelhos' e criar uma nova coluna 'Distrito' no DataFrame Quebras
Quebras['Distrito'] = Quebras['Concelho'].apply(mapear_distrito)
Quebras = Quebras[Quebras['Distrito'] != 'Azores']
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano,Distrito
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,"0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018,Setúbal
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,"0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018,Castelo Branco
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,"0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018,Porto
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,"0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018,Lisboa
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,"0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018,Portalegre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017,Aveiro
7315,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Aveiro
7316,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017,Aveiro
7317,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Santarém


#### 9.1.1 TODO - Não sei o que fazer com estes

In [22]:
Quebras[Quebras["Distrito"].isna()]["Concelho"].unique()

array(['Território De Portugal Continental', 'Portugal Continental',
       'Região Centro De Portugal',
       '(Distritos) Aveiro, Castelo Branco, Portalegre, Porto, Santarém e Viseu',
       'Distritos De Beja e Faro',
       'Distritos De Aveiro, Braga, Bragança, Castelo Branco, Coimbra, Guarda, Porto, Vila Real e Viseu',
       'Distritos De Aveiro, Braga, Castelo Branco, Coimbra, Évora, Faro, Guarda, Leiria, Portalegre, Porto, Viana Do Castelo, Vila Real e Viseu',
       'Tavira, Silves, Loulé, Faro, Serpa, Lagoa, Beja', 'Região Norte',
       'Região Aveiro - Feira', 'Região Leiria - Caldas'], dtype=object)

In [23]:
Concelho[Concelho['Concelho'].str.contains("Vinhos")]

,ID_0,ISO,NAME_0,ID_1,Distrito,ID_2,Concelho,HASC_2,CCN_2,CCA_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,geometry
158,182,PRT,Portugal,11,Leiria,157,Figueiró Dos Vinhos,PT.LE.FV,0,1008,Concelho,Municipality,None,None,"POLYGON ((-8.23894 40.06718, -8.23880 40.06716..."
168,182,PRT,Portugal,12,Lisboa,167,Arruda Dos Vinhos,PT.LI.AV,0,1102,Concelho,Municipality,None,None,"POLYGON ((-9.09276 39.02848, -9.09268 39.02848..."


In [24]:
Quebras = Quebras.dropna(subset=['Distrito'])

In [25]:
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano,Distrito
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,"0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018,Setúbal
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,"0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018,Castelo Branco
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,"0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018,Porto
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,"0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018,Lisboa
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,"0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018,Portalegre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017,Aveiro
7315,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Aveiro
7316,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017,Aveiro
7317,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Santarém


## 9.2. Incêndios - TODO

In [26]:
## Juntar a Base de dados dos incêndios
# Quebras = pd.merge(Quebras, FogoFinal, on=['Ano', 'Mês', 'Concelho'], how='left')
# Quebras

## 9.3. Densidade Populacional

In [27]:
for index, row in Quebras.iterrows():
    # Obtendo o ano e o concelho da linha atual
    ano = row['Ano']
    if ano == 2023:
        ano = 2022
    concelho = row['Concelho']
    
    # Verificando se o ano existe como coluna na tabela 'Denso'
    if ano in Denso.columns:
        # Verificando se o concelho está presente na tabela 'Denso'
        if concelho in Denso['Concelho'].values:
            # Filtrando a linha correspondente na tabela 'Denso' com base no concelho
            linha_denso = Denso[Denso['Concelho'] == concelho]
            
            # Obtendo o valor para o ano desejado
            valor = linha_denso[ano].values[0]
        else:
            # Atribuindo valor nulo se o concelho não estiver presente na tabela 'Denso'
            valor = None
    else:
        # Atribuindo valor nulo se o ano não estiver presente como coluna na tabela 'Denso'
        valor = None
    
    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'Densidade Populacional'] = valor

Quebras


C:\Users\diogo\AppData\Local\Temp\ipykernel_11600\3544803156.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Quebras.at[index, 'Densidade Populacional'] = valor


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano,Distrito,Densidade Populacional
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,"0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018,Setúbal,2516.2
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,"0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018,Castelo Branco,33.1
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,"0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018,Porto,486.5
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,"0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018,Lisboa,198.6
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,"0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018,Portalegre,10.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,"0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017,Aveiro,369.4
7315,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,"0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Aveiro,631.8
7316,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,"0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017,Aveiro,631.8
7317,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,"0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Santarém,100.7


## 9.4. População Residente

In [28]:
for index, row in Quebras.iterrows():
    # Obtendo o ano e o concelho da linha atual
    ano = row['Ano']
    if ano == 2023:
        ano = 2022
    concelho = row['Concelho']
    
    # Verificando se o ano existe como coluna na tabela 'Denso'
    if ano in Viver.columns:
        # Verificando se o concelho está presente na tabela 'Denso'
        if concelho in Viver['Concelho'].values:
            # Filtrando a linha correspondente na tabela 'Denso' com base no concelho
            linha_denso = Viver[Viver['Concelho'] == concelho]
            
            # Obtendo o valor para o ano desejado
            valor = linha_denso[ano].values[0]
        else:
            # Atribuindo valor nulo se o concelho não estiver presente na tabela 'Denso'
            valor = None
    else:
        # Atribuindo valor nulo se o ano não estiver presente como coluna na tabela 'Denso'
        valor = None
    
    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'População Residente'] = valor

Quebras


C:\Users\diogo\AppData\Local\Temp\ipykernel_11600\2221921328.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Quebras.at[index, 'População Residente'] = valor


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano,Distrito,Densidade Populacional,População Residente
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018,Setúbal,2516.2,176158.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018,Castelo Branco,33.1,14792.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018,Porto,486.5,56307.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018,Lisboa,198.6,80859.0
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018,Portalegre,10.9,6256.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017,Aveiro,369.4,54559.0
7315,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Aveiro,631.8,136385.0
7316,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017,Aveiro,631.8,136385.0
7317,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Santarém,100.7,22374.0


## 9.5. Setores de Atividade

### 9.5.1. Primário

In [29]:
for index, row in Quebras.iterrows():
    # Obtendo o ano da linha atual de 'Quebras'
    ano_quebras = row['Ano']
    
    # Obtendo os anos disponíveis na tabela 'WorkP'
    anos_workp = [str(col) for col in WorkP.columns[1:]]  # Convertendo os anos para strings
    
    # Verificando se o ano de 2021 está presente
    if '2021' in anos_workp:
        # Encontrando o ano mais próximo na tabela 'WorkP'
        ano_proximo = min(anos_workp, key=lambda x: abs(int(x) - ano_quebras))
        
        # Obtendo o valor da coluna correspondente ao ano mais próximo
        valor = WorkP[int(ano_proximo)].values[0]
    
    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'Setor Primário'] = valor

Quebras

C:\Users\diogo\AppData\Local\Temp\ipykernel_11600\1253407667.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Quebras.at[index, 'Setor Primário'] = valor


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano,Distrito,Densidade Populacional,População Residente,Setor Primário
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018,Setúbal,2516.2,176158.0,374.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018,Castelo Branco,33.1,14792.0,374.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018,Porto,486.5,56307.0,374.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018,Lisboa,198.6,80859.0,374.0
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018,Portalegre,10.9,6256.0,374.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017,Aveiro,369.4,54559.0,374.0
7315,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Aveiro,631.8,136385.0,374.0
7316,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017,Aveiro,631.8,136385.0,374.0
7317,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Santarém,100.7,22374.0,374.0


### 9.5.2. Secundário

In [30]:
for index, row in Quebras.iterrows():
    # Obtendo o ano da linha atual de 'Quebras'
    ano_quebras = row['Ano']
    
    # Obtendo os anos disponíveis na tabela 'WorkS'
    anos_WorkS = [str(col) for col in WorkS.columns[1:]]  # Convertendo os anos para strings
    
    # Verificando se o ano de 2021 está presente
    if '2021' in anos_WorkS:
        # Encontrando o ano mais próximo na tabela 'WorkS'
        ano_proximo = min(anos_WorkS, key=lambda x: abs(int(x) - ano_quebras))
        
        # Obtendo o valor da coluna correspondente ao ano mais próximo
        valor = WorkS[int(ano_proximo)].values[0]
    
    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'Setor Secundário'] = valor

Quebras

C:\Users\diogo\AppData\Local\Temp\ipykernel_11600\4123524389.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Quebras.at[index, 'Setor Secundário'] = valor


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Fundamentação,Decisão,Dia,Mês,Ano,Distrito,Densidade Populacional,População Residente,Setor Primário,Setor Secundário
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018,Setúbal,2516.2,176158.0,374.0,2486.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018,Castelo Branco,33.1,14792.0,374.0,2486.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018,Porto,486.5,56307.0,374.0,2486.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018,Lisboa,198.6,80859.0,374.0,2486.0
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018,Portalegre,10.9,6256.0,374.0,2486.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017,Aveiro,369.4,54559.0,374.0,2486.0
7315,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Aveiro,631.8,136385.0,374.0,2486.0
7316,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017,Aveiro,631.8,136385.0,374.0,2486.0
7317,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Santarém,100.7,22374.0,374.0,2486.0


### 9.5.3. Terciário

In [31]:
for index, row in Quebras.iterrows():
    # Obtendo o ano da linha atual de 'Quebras'
    ano_quebras = row['Ano']
    
    # Obtendo os anos disponíveis na tabela 'WorkT'
    anos_WorkT = [str(col) for col in WorkT.columns[1:]]  # Convertendo os anos para strings
    
    # Verificando se o ano de 2021 está presente
    if '2021' in anos_WorkT:
        # Encontrando o ano mais próximo na tabela 'WorkT'
        ano_proximo = min(anos_WorkT, key=lambda x: abs(int(x) - ano_quebras))
        
        # Obtendo o valor da coluna correspondente ao ano mais próximo
        valor = WorkT[int(ano_proximo)].values[0]
    
    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'Setor Terciário'] = valor

Quebras

C:\Users\diogo\AppData\Local\Temp\ipykernel_11600\4160773281.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Quebras.at[index, 'Setor Terciário'] = valor


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Decisão,Dia,Mês,Ano,Distrito,Densidade Populacional,População Residente,Setor Primário,Setor Secundário,Setor Terciário
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,Não Aprovado,2,1,2018,Setúbal,2516.2,176158.0,374.0,2486.0,4141.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,Aprovado,2,1,2018,Castelo Branco,33.1,14792.0,374.0,2486.0,4141.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,Aprovado,4,1,2018,Porto,486.5,56307.0,374.0,2486.0,4141.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,Não Aprovado,5,1,2018,Lisboa,198.6,80859.0,374.0,2486.0,4141.0
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,Aprovado,9,1,2018,Portalegre,10.9,6256.0,374.0,2486.0,4141.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,Não Aprovado,29,11,2017,Aveiro,369.4,54559.0,374.0,2486.0,4141.0
7315,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,Não Aprovado,3,11,2017,Aveiro,631.8,136385.0,374.0,2486.0,4141.0
7316,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,Não Aprovado,23,11,2017,Aveiro,631.8,136385.0,374.0,2486.0,4141.0
7317,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,Não Aprovado,3,11,2017,Santarém,100.7,22374.0,374.0,2486.0,4141.0


## 9.6. Indice de Envelhecimento

In [32]:
for index, row in Quebras.iterrows():
    # Obtendo o ano e o concelho da linha atual
    ano = row['Ano']
    if ano == 2023:
        ano = 2022
    concelho = row['Concelho']
    
    # Verificando se o ano existe como coluna na tabela 'Denso'
    if ano in Velho.columns:
        # Verificando se o concelho está presente na tabela 'Denso'
        if concelho in Velho['Concelho'].values:
            # Filtrando a linha correspondente na tabela 'Denso' com base no concelho
            linha_denso = Velho[Velho['Concelho'] == concelho]
            
            # Obtendo o valor para o ano desejado
            valor = linha_denso[ano].values[0]
        else:
            # Atribuindo valor nulo se o concelho não estiver presente na tabela 'Denso'
            valor = None
    else:
        # Atribuindo valor nulo se o ano não estiver presente como coluna na tabela 'Denso'
        valor = None
    
    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'Indice de Envelhecimento'] = valor

Quebras

C:\Users\diogo\AppData\Local\Temp\ipykernel_11600\753980637.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Quebras.at[index, 'Indice de Envelhecimento'] = valor


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Dia,Mês,Ano,Distrito,Densidade Populacional,População Residente,Setor Primário,Setor Secundário,Setor Terciário,Indice de Envelhecimento
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,2,1,2018,Setúbal,2516.2,176158.0,374.0,2486.0,4141.0,157.9
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,2,1,2018,Castelo Branco,33.1,14792.0,374.0,2486.0,4141.0,260.2
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,4,1,2018,Porto,486.5,56307.0,374.0,2486.0,4141.0,117.9
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,5,1,2018,Lisboa,198.6,80859.0,374.0,2486.0,4141.0,156.4
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,9,1,2018,Portalegre,10.9,6256.0,374.0,2486.0,4141.0,467.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,29,11,2017,Aveiro,369.4,54559.0,374.0,2486.0,4141.0,141.7
7315,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,3,11,2017,Aveiro,631.8,136385.0,374.0,2486.0,4141.0,134.6
7316,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,23,11,2017,Aveiro,631.8,136385.0,374.0,2486.0,4141.0,134.6
7317,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,3,11,2017,Santarém,100.7,22374.0,374.0,2486.0,4141.0,175.1


## 9.7. tx Ruralidade

In [33]:
Quebras = Quebras.merge(ZonasRurais[['Concelho', 'Rural']], on='Concelho', how='left')
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Mês,Ano,Distrito,Densidade Populacional,População Residente,Setor Primário,Setor Secundário,Setor Terciário,Indice de Envelhecimento,Rural
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,1,2018,Setúbal,2516.2,176158.0,374.0,2486.0,4141.0,157.9,0.000000
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,1,2018,Castelo Branco,33.1,14792.0,374.0,2486.0,4141.0,260.2,1.000000
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,1,2018,Porto,486.5,56307.0,374.0,2486.0,4141.0,117.9,0.800000
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,1,2018,Lisboa,198.6,80859.0,374.0,2486.0,4141.0,156.4,0.923077
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,1,2018,Portalegre,10.9,6256.0,374.0,2486.0,4141.0,467.9,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,11,2017,Aveiro,369.4,54559.0,374.0,2486.0,4141.0,141.7,1.000000
7287,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,11,2017,Aveiro,631.8,136385.0,374.0,2486.0,4141.0,134.6,0.000000
7288,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,11,2017,Aveiro,631.8,136385.0,374.0,2486.0,4141.0,134.6,0.000000
7289,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,11,2017,Santarém,100.7,22374.0,374.0,2486.0,4141.0,175.1,1.000000


## 9.8. IPMA

In [34]:
# Lista de todas as tabelas de clima
tabelas_clima = tabelas = [('T_MED', T_MED), ('T_MAX', T_MAX), ('T_MIN', T_MIN), ('DD_MED', DD_MED), ('DD_FFX', DD_FFX), ('FF_MED', FF_MED), ('FF_MAX', FF_MAX), ('PR_QTD', PR_QTD)]

# Iterar sobre todas as tabelas de clima
for tabela in tabelas_clima:
    # Obtendo o nome da tabela
    nome_tabela = tabela[0]
    TabeIPMA = tabela[1]
    # Iterar sobre as linhas de Quebras
    for index, row in Quebras.iterrows():

        ano = int(row['Ano'])
        mes = int(row['Mês'])
        dia = int(row['Dia'])
        concelho = row["Distrito"]

        
        valor = TabeIPMA[concelho][(TabeIPMA["Ano"] == ano) & (TabeIPMA["Mês"] == mes) & (TabeIPMA["Dia"] == dia)].iloc[0]
        
        # Atualizando a linha 'Quebras' com o valor obtido
        Quebras.at[index, f'{nome_tabela}'] = valor

# Exibindo o DataFrame Quebras atualizado
Quebras


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Indice de Envelhecimento,Rural,T_MED,T_MAX,T_MIN,DD_MED,DD_FFX,FF_MED,FF_MAX,PR_QTD
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,157.9,0.000000,10.7,17.4,2.6,-990.0,-990.0,-990.0,-990.0,0.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,260.2,1.000000,10.6,13.8,6.2,249.0,270.0,2.8,8.1,0.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,117.9,0.800000,14.0,14.8,12.9,197.0,180.0,3.9,14.4,4.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,156.4,0.923077,12.4,15.1,9.1,288.0,225.0,3.6,13.9,13.2
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,467.9,1.000000,4.9,7.8,2.2,137.0,118.0,4.1,15.4,15.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,141.7,1.000000,10.3,14.3,6.3,-990.0,-990.0,-990.0,-990.0,0.0
7287,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,134.6,0.000000,16.4,20.6,14.8,-990.0,-990.0,-990.0,-990.0,-990.0
7288,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,134.6,0.000000,17.9,22.6,14.6,-990.0,-990.0,-990.0,-990.0,8.1
7289,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,175.1,1.000000,15.0,20.8,12.8,186.0,186.0,2.5,11.0,-990.0


In [35]:
Quebras.replace(-990.0, None, inplace=True)
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Indice de Envelhecimento,Rural,T_MED,T_MAX,T_MIN,DD_MED,DD_FFX,FF_MED,FF_MAX,PR_QTD
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828.0,"0,0029345","0,0000443","0,0024563",...,157.9,0.000000,10.7,17.4,2.6,None,None,None,None,0.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549.0,"0,0029335","0,0001770","0,0165009",...,260.2,1.000000,10.6,13.8,6.2,249.0,270.0,2.8,8.1,0.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459.0,"0,0192281","0,0002655","0,0359047",...,117.9,0.800000,14.0,14.8,12.9,197.0,180.0,3.9,14.4,4.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102.0,"0,0000000","0,0000000","0,0000000",...,156.4,0.923077,12.4,15.1,9.1,288.0,225.0,3.6,13.9,13.2
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731.0,"0,0044221","0,0001549","0,0100377",...,467.9,1.000000,4.9,7.8,2.2,137.0,118.0,4.1,15.4,15.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2.0,0,0,0,...,141.7,1.000000,10.3,14.3,6.3,None,None,None,None,0.0
7287,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1.0,0,0,0,...,134.6,0.000000,16.4,20.6,14.8,None,None,None,None,None
7288,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1.0,0,0,0,...,134.6,0.000000,17.9,22.6,14.6,None,None,None,None,8.1
7289,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1.0,0,0,0,...,175.1,1.000000,15.0,20.8,12.8,186.0,186.0,2.5,11.0,None


In [39]:
Quebras.to_csv("QuebrasJuntoInc.csv")

In [36]:
len(Quebras.columns)

36

In [37]:
QuebrasFinal = Quebras.copy()

In [38]:
kk

NameError: name 'kk' is not defined

In [ ]:
# Selecionar apenas as colunas numéricas
colunas_numericas = QuebrasFinal.select_dtypes(include=['float64', 'int64']).columns

# 1. Estandardização
scaler = StandardScaler()
QuebrasFinal_scaled = scaler.fit_transform(QuebrasFinal[colunas_numericas])

# 2. Imputação por Regressão
imputer = IterativeImputer()
QuebrasFinal_imputed = imputer.fit_transform(QuebrasFinal_scaled)

# Converter de volta para DataFrame, se necessário
QuebrasFinal_imputed = pd.DataFrame(QuebrasFinal_imputed, columns=colunas_numericas)

QuebrasFinal_imputed

In [ ]:
sns.set(style="whitegrid")

# Plotar violin plot para todas as colunas
plt.figure(figsize=(16, 10))
sns.violinplot(data=QuebrasFinal_imputed, inner="points", orient="h")
plt.title("Violin Plot para Todas as Colunas")
plt.xlabel("Valores")
plt.ylabel("Colunas")
plt.show()

In [ ]:
QuebrasFinal = QuebrasFinal[QuebrasFinal["aprovado"]==0]

In [ ]:
# 1. Estandardização
scaler = StandardScaler()
QuebrasFinal_scaled = scaler.fit_transform(QuebrasFinal)

# 2. Imputação por Regressão
imputer = IterativeImputer()
QuebrasFinal_imputed = imputer.fit_transform(QuebrasFinal_scaled)

# Converter de volta para DataFrame, se necessário
QuebrasFinal_imputed = pd.DataFrame(QuebrasFinal_imputed, columns=QuebrasFinal.columns)

QuebrasFinal_imputed

In [ ]:
sns.set(style="whitegrid")

# Plotar violin plot para todas as colunas
plt.figure(figsize=(16, 10))
sns.violinplot(data=QuebrasFinal_imputed, inner="points", orient="h")
plt.title("Violin Plot para Todas as Colunas")
plt.xlabel("Valores")
plt.ylabel("Colunas")
plt.savefig("Apagar.png")
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Valores de k que você deseja testar
k_values = range(2, 7)

# Lista para armazenar os valores de silhueta média
silhouette_scores = []

# Iterar sobre os valores de k
for k in k_values:
    # Criar o modelo de clustering K-Means
    kmeans = KMeans(n_clusters=k, random_state=42)
    # Treinar o modelo nos dados
    kmeans.fit(QuebrasFinal_imputed)
    # Obter as etiquetas de cluster para cada ponto
    labels = kmeans.labels_
    # Calcular o score de silhueta
    silhouette_avg = silhouette_score(QuebrasFinal_imputed, labels)
    # Armazenar o score de silhueta na lista
    silhouette_scores.append(silhouette_avg)

# Plotar o gráfico do Método do Cotovelo
plt.plot(k_values, silhouette_scores, 'bx-')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Silhueta Média')
plt.title('Método da Silhueta para Determinar k')
plt.show()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Reduzir a dimensionalidade dos dados para 2 dimensões usando t-SNE
tsne = TSNE(n_components=2, random_state=69)
data_2d = tsne.fit_transform(QuebrasFinal_imputed)

# Plotar os pontos em um gráfico de dispersão, colorindo-os de acordo com os clusters
plt.scatter(data_2d[:, 0], data_2d[:, 1], c=labels, cmap='viridis')
plt.title('Visualização dos Clusters com t-SNE')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import make_blobs
from sklearn.cluster import AgglomerativeClustering

# Gerar dados de exemplo
X, _ = make_blobs(n_samples=150, centers=5, random_state=42)

# Executar o algoritmo de clustering hierárquico
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
model.fit(X)

# Função para plotar o dendrograma
def plot_dendrogram(model, **kwargs):
    # Calcula o layout do dendrograma
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # folha
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)

    # Plota o dendrograma
    dendrogram(linkage_matrix, **kwargs)

# Plotando o dendrograma
plt.figure(figsize=(10, 5))
plt.title('Dendrograma Hierárquico')
plot_dendrogram(model)
plt.xlabel('Índice do Amostra')
plt.ylabel('Distância')
plt.grid(False)
plt.savefig("Apagar.png")
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Inicialize o modelo Agglomerative Clustering com o número desejado de clusters
n_clusters = 3  # Defina o número de clusters desejado
agg_clustering = AgglomerativeClustering(n_clusters=n_clusters)

# Ajuste o modelo aos dados
agg_clustering.fit(QuebrasFinal_imputed)

# Obtenha as etiquetas de cluster para cada amostra
cluster_labels = agg_clustering.labels_

# Adicione as etiquetas de cluster ao DataFrame QuebrasFinal_imputed
QuebrasFinal_imputed['Cluster_Agglomerative'] = cluster_labels

In [ ]:
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.pyplot as plt
import numpy as np

# Calcule a silhouette média para o modelo
silhouette_avg = silhouette_score(QuebrasFinal_imputed, cluster_labels)
print("Silhouette média para o modelo Agglomerative Clustering:", silhouette_avg)

# Calcule a silhouette para cada amostra
sample_silhouette_values = silhouette_samples(QuebrasFinal_imputed, cluster_labels)

# Crie um gráfico de barras para mostrar a silhouette para cada amostra
plt.figure(figsize=(10, 6))
y_lower = 10
for i in range(n_clusters):
    # Agregue a silhouette para amostras pertencentes ao cluster i
    ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
    ith_cluster_silhouette_values.sort()
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = plt.cm.nipy_spectral(float(i) / n_clusters)
    plt.fill_betweenx(np.arange(y_lower, y_upper),
                      0, ith_cluster_silhouette_values,
                      facecolor=color, edgecolor=color, alpha=0.7)

    # Etiqueta do cluster no meio
    plt.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    # Calcule o próximo y_lower para o próximo gráfico
    y_lower = y_upper + 10  

plt.title("Gráfico de Silhouette para Agglomerative Clustering")
plt.xlabel("Valor de silhouette")
plt.ylabel("Cluster")
plt.axvline(x=silhouette_avg, color="red", linestyle="--")  # Linha para indicar a silhouette média
plt.yticks([])  # Limpe os yticks
plt.show()


In [ ]:
QuebrasFinal_imputed

In [ ]:
# Agrupar os dados por cluster e calcular as médias
cluster_means = QuebrasFinal_imputed.groupby('Cluster_Agglomerative').mean()

# Definir cores para os clusters
colors = ['blue', 'green', 'orange', 'red']

# Plotar os gráficos de barras para cada cluster
for i, cluster in enumerate(cluster_means.index):
    cluster_data = cluster_means.loc[cluster]
    plt.figure(figsize=(10, 6))
    plt.barh(cluster_data.index, cluster_data.values, color=colors[i])
    plt.title(f'Médias das Variáveis para o Cluster {cluster}')
    plt.xlabel('Média')
    plt.ylabel('Variável')
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.show()

In [ ]:
# Supondo que 'clusters' seja a coluna de clusters em QuebrasFinal_imputed
clusters = QuebrasFinal_imputed['Cluster_Agglomerative']

# Adicionando a coluna de clusters ao DataFrame Quebras
Quebras['Cluster_Agglomerative'] = clusters

In [ ]:
# Agrupar os dados por causa do incidente e cluster e contar o número de ocorrências em cada grupo
causa_cluster_counts = Quebras.groupby(['causa_do_incidente', 'Cluster_Agglomerative']).size().unstack(fill_value=0)

# Plotar o gráfico de barras empilhadas para cada causa do incidente
causa_cluster_counts.plot(kind='bar', stacked=True, figsize=(16, 12))
plt.title('Quantidade de Incidentes por Causa e Cluster')
plt.xlabel('Causa do Incidente')
plt.ylabel('Quantidade')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Cluster')
plt.show()

In [ ]:
# Agrupar os dados por 'nt' e 'Cluster_Agglomerative' e contar o número de ocorrências em cada grupo
nt_cluster_counts = Quebras.groupby(['nt', 'Cluster_Agglomerative']).size().unstack(fill_value=0)

# Calcular a porcentagem de ocorrências de cada cluster para cada valor de 'nt'
nt_cluster_percentages = nt_cluster_counts.div(nt_cluster_counts.sum(axis=1), axis=0) * 100

# Plotar gráficos circulares para a porcentagem de clusters em cada valor de 'nt'
for nt_value, nt_cluster_percentage in nt_cluster_percentages.iterrows():
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.pie(nt_cluster_percentage, labels=nt_cluster_percentage.index, autopct='%1.1f%%', startangle=90)
    ax.set_title(f'Porcentagem de Clusters em "nt" = {nt_value}')
    plt.tight_layout()
    plt.savefig(f'Porcentagem_Clusters_nt_{nt_value}.svg')
    plt.show()


In [ ]:
# Criar a tabela pivô
pivot_table = Quebras.pivot_table(index='Concelho', columns='Cluster_Agglomerative', aggfunc='size', fill_value=0)

# Renomear as colunas
pivot_table.columns = ['Cluster_' + str(col) for col in pivot_table.columns]

# Exibir a tabela pivô
pivot_table
pivot_table['Cluster_2.0'].idxmax()

In [ ]:
file_path = shape_folder / "concelhos.shp"
Concelho = gpd.read_file(file_path)
Concelho.rename(columns={'NAME_1': 'Distrito'}, inplace=True)
Concelho.rename(columns={'NAME_2': 'Concelho'}, inplace=True)
Concelho["Concelho"] = Concelho["Concelho"].str.title()
Concelho = Concelho[~Concelho['Distrito'].isin(['Azores', 'Madeira'])]
Concelho

In [ ]:
Bilada = set(pivot_table.index) - set(Concelho["Concelho"])
Bilada

In [ ]:
Freguesias_com_clusters = Concelho.merge(pivot_table, left_on='Concelho', right_index=True, how="left")
Freguesias_com_clusters.fillna(0, inplace=True)
Freguesias_com_clusters

In [ ]:
# Configurar a figura e os eixos para a grade de subplots
fig, axs = plt.subplots(1, 3, figsize=(20, 8))

# Iterar sobre cada cluster e plotar o mapa correspondente
for i, cluster in enumerate(['Cluster_0.0', 'Cluster_1.0', 'Cluster_2.0']):
    # Plotar o mapa das freguesias para o cluster atual
    Freguesias_com_clusters.plot(column=cluster, cmap='RdYlGn_r', linewidth=0.8, edgecolor='black', ax=axs[i], legend=True)
    
    # Adicionar título para cada subplot
    axs[i].set_title(f'Mapa do Cluster {i}', fontsize=14)

# Exibir os mapas
plt.tight_layout()
plt.show()


In [ ]:
# Configurar a figura e o eixo
fig, ax = plt.subplots(figsize=(16, 14))

# Plotar o mapa das freguesias
Freguesias_com_clusters.plot(column='Cluster_0.0', cmap = "RdYlGn_r", linewidth=0.8, edgecolor='black', ax=ax, legend=True)

# Adicionar título
ax.set_title('Zonas das Freguesias', fontsize=16)

# Exibir o mapa
plt.show()

In [ ]:
# Configurar a figura e o eixo
fig, ax = plt.subplots(figsize=(16, 14))

# Plotar o mapa das freguesias
Freguesias_com_clusters.plot(column='Cluster_1.0', cmap = "RdYlGn_r", linewidth=0.8, edgecolor='black', ax=ax, legend=True)

# Adicionar título
ax.set_title('Zonas das Freguesias', fontsize=16)

# Exibir o mapa
plt.show()

In [ ]:
# Configurar a figura e o eixo
fig, ax = plt.subplots(figsize=(16, 14))

# Plotar o mapa das freguesias
Freguesias_com_clusters.plot(column='Cluster_2.0', cmap = "RdYlGn_r", linewidth=0.8, edgecolor='black', ax=ax, legend=True)

# Adicionar título
ax.set_title('Zonas das Freguesias', fontsize=16)

plt.savefig("Apagar.png")

# Exibir o mapa
plt.show()